In [ ]:
import numpy as np
import random
import math
from copy import deepcopy

import pandas as pd

%matplotlib notebook

# Milestone
The instructor script will perform a Monte Carlo simulation using the following user defined variables:
- reduced temperature (reduced_temperature)
- cut-off (cutoff) The cut-off for LJ energy calculation
- number of moves (num_steps) - The number of displacements to try
- maximum displacement (max_displacement) - The maximum displacement of a particle in one dimension during a move.

In [ ]:
# Parameters
reduced_temperature = 0.9
num_steps = 100
max_displacement = 1
cutoff = 3.0

# Calculated quantities
beta = 1 / reduced_temperature


# Milestone
The instructor script will read an initial configuration from an xyz file from NIST.

In [ ]:
# Get initial coordinates and box info. Units are in reduced units.
import os

filename = os.path.join('..', 'data', 'sample_config1.xyz')
# Coordinates are columns 1,2,3 in xyz file
coordinates = np.loadtxt(filename, skiprows=2, usecols=(1,2,3))

# Atom names is column 0 in xyz file.
atoms = np.loadtxt(filename, skiprows=2, usecols=[0], dtype='str')

# The second line in the xyz file has the box size for periodic boundaries.
with open(filename) as f:
    f.readline()
    box_info = [float(x) for x in f.readline().split()]

# The box length for the simulation is set to the maximum of the box size.
box_length = np.max(box_info)

# The number of particles is determined from the number of particles in the initial configuration.
num_particles = len(atoms)

# Milestone
**Accuracy Check**: The instructor script will calculate the energy of the initial configuration of the system, and will match the energy given by NIST for the configuration. https://www.nist.gov/mml/csd/chemical-informatics-research-group/lennard-jones-fluid-reference-calculations
- U_pair energy
- U_LRC tail correction


In [ ]:
## Determine the total system pairwise energy
total_energy = 0

for i, position_i in enumerate(coordinates):
    for j, position_j in enumerate(coordinates):
        if j > i:
            rij = position_i - position_j
            rij = rij - box_length * np.round(rij/box_length)
            rij = np.sqrt(np.dot(rij, rij))

            if rij < cutoff:
                # If less than cut off, calculate pairwise energy
                r6_term = np.power(1/rij, 6)
                r12_term = np.power(r6_term, 2)
                pairwise_energy = 4.0 * (r12_term - r6_term)
                total_energy += pairwise_energy
print(F"The total system pairwise energy is {total_energy}")
total_energy = round(total_energy,1)
# Assert that we match the NIST value.
assert total_energy == -4351.5

In [ ]:
## Long range tail correction
tail_correction = 8*np.pi*num_particles**2/(3*box_length**3)*((1/3)*(1/cutoff)**9 - (1/cutoff)**3)
tail_correction = round(tail_correction,2)

print(F'Internal energy tail correction {tail_correction}')

assert tail_correction == -1.9849E+02

# Milestones
1. The instructor script will implement a Monte Carlo simulation of a Lennard Jones fluid.
1. The instructor script will be a flat script with no user defined functions.
1. The instructor script will use reduced units.
1. A cut-off will be used for LJ calculation - that is, if particles are separated by a distance greater than the cut-off distance, the LJ interaction will not be calculated.

In [ ]:
"""
Note to instructor - This was written by first writing the comments (#) about what the program would do, then code
was filled in for each step.
"""

# Loop over steps

# Make a copy of the coordinates so we keep the original
initial_coordinates = deepcopy(coordinates)
delta_energy = 0

for step in range(num_steps):

    # Randomly pick one of N particles
    random_particle = random.randrange(num_particles)
    
    # Generate random x, y, z displacement
    x_rand = random.uniform(-max_displacement, max_displacement)
    y_rand = random.uniform(-max_displacement, max_displacement)
    z_rand = random.uniform(-max_displacement, max_displacement)
    
    # Modify coordinate of Nth particle by generated displacements
    original_coordinates = coordinates[random_particle, :]
    new_coordinates = deepcopy(original_coordinates)
    
    new_coordinates[0] += x_rand
    new_coordinates[1] += y_rand
    new_coordinates[2] += z_rand
    
    new_energy = 0
    old_energy = 0
    
    # Compute energy difference from particle movement. 
    
    for i_particle in range(0, num_particles):
       
        if i_particle != random_particle:
            # Compute original energy of particle with system.
            
            # Compute distance between particles using minimum image convention
            r_i = coordinates[i_particle, :]
            r_j = original_coordinates
            rij = r_i - r_j
            rij = rij - box_length * np.round(rij/box_length)
            rij2 = np.sqrt(np.dot(rij, rij))

            # Check if distance is less than the cut-off
            if rij2 < cutoff:
                # If less than cut off, calculate pairwise energy
                sig_by_r6 = np.power(1/rij2, 6)
                sig_by_r12 = np.power(sig_by_r6, 2)
                original_pairwise_energy = 4.0 * (sig_by_r12 - sig_by_r6)
            else:
                # Else, pairwise energy is 0
                original_pairwise_energy = 0
            
            old_energy += original_pairwise_energy

            # Compute new energy of particle with system.

            # Compute distance between particles using minimum image convention
            r_j = new_coordinates
            rij = r_i - r_j
            rij = rij - box_length * np.round(rij/box_length)
            rij2 = np.sqrt(np.dot(rij, rij))

            # Check if distance is less than the cut-off
            if rij2 < cutoff:
                # If less than cut off, calculate pairwise energy
                sig_by_r6 = np.power(1/rij2, 6)
                sig_by_r12 = np.power(sig_by_r6, 2)
                new_pairwise_energy = 4.0 * (sig_by_r12 - sig_by_r6)
            else:
                # Else, pairwise energy is 0
                new_pairwise_energy = 0
            
            new_energy += new_pairwise_energy
        
    # Calculate change in potential energy for particle move.
    delta_energy = new_energy - old_energy
    
    # If negative or 0 change in energy, accept move, overwrite coordinates
    # if deltaU >0 check acceptance criteria
    if delta_energy <= 0:
        coordinates[random_particle, :] = new_coordinates
        total_energy += delta_energy
    else:
        probability_acc = math.exp(-delta_energy*beta)
        rand_val = random.uniform(0, 1)
        
        if probability_acc > rand_val:
            # Move is accepted.
            coordinates[random_particle, :] = new_coordinates
            total_energy += delta_energy
        else:
            # Move is not accepted. Do not update coordinates
            pass
    
    
    print(step, total_energy)
        